<font color = 'stone Blue'> Work on this version of the draft started again on April 1, 2023
</font>

# Part 1: Data download, and preprocessing
    1. Download: MODIS Data (2001-2020)-- Done
    2. Prepare geemap interface
        1. add basemap--Done
        2. crop area for faster processing--Done
        3. export into a directory and export geoTIFF results in hard drive-- Done.

# Part 2: Timeseries analysis 
# Part 3: Landcover change by region
1. Need help with dividing the GAUL dataset into different districts based on basemap.-- Done
2. Add interactive maps. include timeline. 

# Part 4: Link Ecosystem Services and Landcover change. Qualitative data to be collected. Details here
                

In [1]:
import os
os.getcwd()

'/Users/rishi/1LandCoverIndia'

In [2]:
##############Part 1#############
from osgeo import gdal, gdal_array
import numpy as np
import sys
import matplotlib.pyplot as plt
import rasterio
import fiona
import os
#ee.Authenticate()


In [3]:
#gee features import
import geemap
import ee
ee.Initialize()

## Introduction
   The study area is the National Capital Region (NCR) of India. the region, although according to the office of census operations, a total of 8 districts make up NCR. However, in day-to-day, non legal parlance, not all of these eight districts is used to represent NCR. Coloquially, New delhi, along with Faridabad, Gurgaon, Noida and Ghaziabad are together counted under the umbrealla category of "NCR". 
    *We look at the landcover change in NCR since 2000 using MODIS MCD12Q1 databse.*


In [4]:
import geemap

# Create a Map
Map = geemap.Map(center=[13.4000, 122.5621], zoom=6)  # Centered on the Philippines

# Load administrative boundary feature for the Philippines
selected = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(
    ee.Filter.eq("ADM1_NAME", 'Central Luzon').Or(
    ee.Filter.eq("ADM1_NAME", 'Calabarzon').Or(
    ee.Filter.eq("ADM1_NAME", 'Mimaropa')))
)

# Define the bounding box coordinates for the selected regions
coords = ([119.3226, 12.5621], [125.0420, 12.5621], [125.0420, 18.5822], [119.3226, 18.5822])
selected2 = ee.Geometry.BBox(119.3226, 12.5621, 125.0420, 18.5822)

# Center the map on the selected regions
Map.centerObject(selected2, 7)

# Add the study area layer to the map
Map.addLayer(selected2, {}, 'Study Area')

# Add layer control to the map
Map.addLayerControl()

# Display the map
Map


Map(center=[15.563602988929068, 122.18229999999997], controls=(WidgetControl(options=['position', 'transparent…

Now we download the landcover file for the specified study area for the years 2000-2020 from MODIS Landcover dataset.
We also add the administrative layers to see the district wise change in landcover over these years

In [5]:
#geoTIFFFile upload
file= '/Users/rishi/Downloads/LC_Delhi_2018/StudyArea_Classification2018.tif'


#This 'file' has all data for all the years 
ds= gdal.Open(file)

# Allocate our array using the first band's datatype
ds_datatype=ds.GetRasterBand(1).DataType

image = np.zeros((ds.RasterYSize, ds.RasterXSize, ds.RasterCount),
                 dtype=gdal_array.GDALTypeCodeToNumericTypeCode(ds_datatype))

#Loop over all bands
for b in range(ds.RasterCount):
    band=ds.GetRasterBand(b+1)
    image[:,:,b]=band.ReadAsArray()



In [6]:
#Another way to upload data. By creating a Final Results Folder and using API for download
out_dir = os.path.join(os.path.expanduser("~"), '5LC_Data_Aug2023_MM')
filepath= os.path.join(out_dir, 'DownloadLCgeoTIFF/LC20012020.geoTIFF')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
Map.add_basemap('Esri.NatGeoWorldMap')
# Map = emap.Map(center=[28.668901,77.258606], zoom=8)

# selected = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(
#     ee.Filter.eq("ADM0_NAME", 'India')
# )

#Smaller study area for quicker loading of maps
# Define the bounding box coordinates for the selected regions
coords = ([119.3226, 12.5621], [125.0420, 12.5621], [125.0420, 18.5822], [119.3226, 18.5822])
selected2 = ee.Geometry.BBox(119.3226, 12.5621, 125.0420, 18.5822)

# C
# Center the map on the selected regions
Map.centerObject(selected2, 7)

# Add the study area layer to the map
Map.addLayer(selected2, {}, 'Study Area')

# Add layer control to the map
Map.addLayerControl()

# Display the map
Map
#MODIS is the collection of all the images from MCD12Q1. For all the years. In the next cell, we extract the years we want data for. 

Map(bottom=15250.0, center=[15.563602988929068, 122.18229999999997], controls=(WidgetControl(options=['positio…

In [39]:
# Import required libraries
import os
import geemap

# Define the output directory
out_dir = os.path.join(os.path.expanduser("~"), '5LC_Data_Aug2023_MM')

# Create a Map
Map = geemap.Map(center=[12.8797, 121.7740], zoom=6)  # Centered on the Philippines

# ... (Your code for adding layers and setting up the map)

# Save the map as a PNG image in the specified directory
out_image_path = os.path.join(out_dir, 'map_image.png')
Map.save(out_image_path)

# Display the map
Map


Map(center=[12.8797, 121.774], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

### Download Images from the five time periods as ImgCollection in directory


In [41]:
import pandas as pd

# Your existing data
data = {
    'Ecosystem Condition': ['A1. PHYSICAL STATE', 'A1. PHYSICAL STATE', 'A1. PHYSICAL STATE', 'A1. PHYSICAL STATE',
                             'A2. CHEMICAL STATE', 'A2. CHEMICAL STATE', 'A2. CHEMICAL STATE', 'B1. COMPOSITIONAL STATE',
                             'B1. COMPOSITIONAL STATE', 'B1. COMPOSITIONAL STATE', 'B2. STRUCTURAL STATE', 'B2. STRUCTURAL STATE',
                             'B2. STRUCTURAL STATE', 'B2. STRUCTURAL STATE', 'B3. FUNCTIONAL STATE', 'B3. FUNCTIONAL STATE'],
    'Variable': ['Imperviousness per inhabitant', 'Waste generated per inhabitant', 'Normalized Difference Moisture Index (NDMI)',
                 'Noise pollution exposure', 'Air pollutants concentration (NO2, PMx, O3, SO2, CO)', 'Soil organic carbon stock',
                 'Heavy metals in soil', 'Autochthonous woody vegetation species (or functional trait) richness',
                 'Urban bird species richness', 'Pressure by invasive alien species on urban ecosystems', 'Greenness - annual max NDVI',
                 'Tree canopy cover', 'Green spaces and/or green spaces per inhabitant', 'Semi-natural and natural riparian land cover',
                 'Plant evapotranspiration', 'Wild pollinators indicator'],
    'Units': ['ha/inhabitant', 'kg waste/ inhabitant', 'Dimensionless', 'Inhabitants', 'μg/m3', 'kg C/ha', 'μg/g', 'Number of species (traits)',
              'Number of species', 'Dimensionless', 'Dimensionless', '%', '% or ha/inhabitant', '%', 'mm d-1', 'Dimensionless']
}

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Add a column for the source of remotely sensed data
df['Source from Remotely Sensed Data'] = [
    'Built-Up Area/Population', None, 'LANDSAT', None, 'Annual AQ Statistics', 'Secondary Literature', 'Secondary Literature/ Empirical Studies',
    None, None, None, 'LANDSAT', 'LANDSAT', 'LANDSAT', 'LANDSAT', 'LANDSAT', None
]

# Display the DataFrame
print(df)
# Export the DataFrame to a CSV file
df.to_csv('ecosystem_condition_data.csv', index=False)


        Ecosystem Condition  \
0        A1. PHYSICAL STATE   
1        A1. PHYSICAL STATE   
2        A1. PHYSICAL STATE   
3        A1. PHYSICAL STATE   
4        A2. CHEMICAL STATE   
5        A2. CHEMICAL STATE   
6        A2. CHEMICAL STATE   
7   B1. COMPOSITIONAL STATE   
8   B1. COMPOSITIONAL STATE   
9   B1. COMPOSITIONAL STATE   
10     B2. STRUCTURAL STATE   
11     B2. STRUCTURAL STATE   
12     B2. STRUCTURAL STATE   
13     B2. STRUCTURAL STATE   
14     B3. FUNCTIONAL STATE   
15     B3. FUNCTIONAL STATE   

                                             Variable  \
0                       Imperviousness per inhabitant   
1                      Waste generated per inhabitant   
2         Normalized Difference Moisture Index (NDMI)   
3                            Noise pollution exposure   
4   Air pollutants concentration (NO2, PMx, O3, SO...   
5                           Soil organic carbon stock   
6                                Heavy metals in soil   
7   Autochthonou

In [8]:
## These images need to be corrected for atmospheric disturbances. Maybe take the image which is least distorted?
MODIS2001 = ee.Image('MODIS/006/MCD12Q1/2001_01_01').select('LC_Type1')
MODIS2005 = ee.Image('MODIS/006/MCD12Q1/2005_01_01').select('LC_Type1')
MODIS2011 = ee.Image('MODIS/006/MCD12Q1/2011_01_01').select('LC_Type1')
MODIS2015 = ee.Image('MODIS/006/MCD12Q1/2015_01_01').select('LC_Type1')
MODIS2019 = ee.Image('MODIS/006/MCD12Q1/2019_01_01').select('LC_Type1')

MODIS2 = ee.ImageCollection([MODIS2001, MODIS2005,  MODIS2011,  MODIS2015, MODIS2019]).filterBounds(selected2)

MODIS2_layer_names = ['MODIS2 ' + str(year) for year in range(2001, 2020, 3)]
print(MODIS2_layer_names)

first_image = MODIS2.first()
type(selected2)
geemap.download_ee_image_collection(collection=MODIS2, 
                                    out_dir=out_dir, 
                                    #filenames= "MODIS2_layer_names", 
                                    region= selected2,
                                    scale=30)




['MODIS2 2001', 'MODIS2 2004', 'MODIS2 2007', 'MODIS2 2010', 'MODIS2 2013', 'MODIS2 2016', 'MODIS2 2019']
Total number of images: 5



2001_01_01.tif: |          | 0.00/740M (raw) [  0.0%] in 00:00 (eta:     ?)

2005_01_01.tif: |          | 0.00/740M (raw) [  0.0%] in 00:00 (eta:     ?)

2011_01_01.tif: |          | 0.00/740M (raw) [  0.0%] in 00:00 (eta:     ?)

2015_01_01.tif: |          | 0.00/740M (raw) [  0.0%] in 00:00 (eta:     ?)

2019_01_01.tif: |          | 0.00/740M (raw) [  0.0%] in 00:00 (eta:     ?)

### Add Administrative Boundaries for districts and states within the study area

In [1]:
#define fvLayer-- State Level Boundaries
fvLayer = ee.FeatureCollection("FAO/GAUL/2015/level1").filterBounds(selected2)
vis_params = {
    'min': 0,
    'max': 2000,
    'color':'black',
    'opacity':0.4
  #'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
}


#define fvLayer2-- District Level Boundaries
fvLayer2 = ee.FeatureCollection("FAO/GAUL/2015/level2").filterBounds(selected2)
vis_params2 = {
    'min': 0,
    'max': 2000,
    'color':"purple",
    'opacity':0.8
  #'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
}


NameError: name 'ee' is not defined

#### Previewing first image with state and district administrative boundaries

In [10]:
#Add center to India here: Done(77.24,28.65, zoom=8.5)
#Add Labels to districts
first_image = MODIS2.first().clip(selected2)
fvLayer= fvLayer
vis = {
    'bands': ['LC_Type1'],
    'min': 1.0,
    'max': 17.0,
    'palette': ['05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff']}
Scale=500

vis_params2 = {
    'min': 0,
    'max': 2000,
    #'color':"yellow",
    'opacity':0.9,
  #'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
}

Map2=geemap.Map()
Map2.add_basemap('Esri.NatGeoWorldMap')
Map2.addLayer(first_image, vis, 'LC Classification')
Map2.addLayer(fvLayer, vis_params, 'states')
Map2.addLayer(fvLayer2.filterBounds(selected2), vis_params2, 'Districts')
Map2.center_object(selected2, 6)
#Map2.add_legend(builtin_legend='MODIS/051/MCD12Q1')
Map2.add_layer_control()
#Map2.add_labels(fvLayer2.aggregate_array(fvLayer2['ADM2_NAME']))
Map2

Map(center=[15.563602988929068, 122.18229999999997], controls=(WidgetControl(options=['position', 'transparent…

In [14]:
landcover_names = {
    "1": 'Evergreen Needleleaf Forests',
    "2": 'Evergreen Broadleaf Forests',
    "3": 'Deciduous Needleleaf Forests',
    "4": 'Deciduous Broadleaf Forests',
    "5": 'Mixed Forests',
    "6": 'Closed Shrublands',
    "7": 'Open Shrublands',
    "8": 'Woody Savannas',
    "9": 'Savannas',
    "10": 'Grasslands',
    "11": 'Permanent Wetlands',
    "12": 'Croplands',
    "13": 'Urban and Built-up Lands',
    "14": 'Cropland/Natural Vegetation Mosaics',
    "15": 'Permanent Snow and Ice',
    "16": 'Barren',
    "17": 'Your Landcover Name for 17'
}


In [18]:
# Initialize an empty list to store the data
data = []

# Iterate through the district features
for district in fvLayer2.getInfo()['features']:
    # Extract district and parent region names
    district_name = district['properties']['ADM2_NAME']
    parent_region_name = district['properties']['ADM1_NAME']
    
    # Get the pixel area for the district
    pixel_count = district['properties']['Shape_Area']
    district_area_sq_km = pixel_count * pixel_area
    
    # Get the land cover classification values for the district
    landcover_values = first_image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=district['geometry'],
        scale=Scale,
        maxPixels=1e13
    ).get('LC_Type1').getInfo()

    # Convert pixel counts to landcover areas in square kilometers
    landcover_areas = {landcover_names[str(landcover)]: count * pixel_area for landcover, count in landcover_values.items()}

    # Append the data to the list
    data.append({
        'Parent Region': parent_region_name,
        'District': district_name,
        **landcover_areas,
        'Area (sq km)': district_area_sq_km
    })

# Create a DataFrame from the list of data
district_landcover_df = pd.DataFrame(data)

# Print the resulting DataFrame
print(district_landcover_df)


NameError: name 'pixel_area' is not defined

Cordillera Administrative Region (CAR) and the National Capital Region (NCR). Each row corresponds to a district, and the columns represent various land cover categories, such as "Grasslands." The numbers in each cell indicate the area (in square kilometers) covered by the specific land cover category within that district.

In [16]:
import os

# Export the DataFrame to a CSV file
csv_file_path = '/Users/rishi/5LC_Data_Aug2023_MM/district_landcover.csv'
district_landcover_df.to_csv(csv_file_path, index=False)

# Get the current working directory
current_directory = os.getcwd()

# Print the full path of the saved CSV file
print("CSV file saved at:", os.path.join(current_directory, csv_file_path))



NameError: name 'district_landcover_df' is not defined

In [17]:
# Initialize an empty list to store the data
data = []

# List of time periods (years)
time_periods = [2001, 2005, 2010, 2015, 2020]

# Iterate through the district features
for district in fvLayer2.getInfo()['features']:
    # Extract district and parent region names
    district_name = district['properties']['ADM2_NAME']
    parent_region_name = district['properties']['ADM1_NAME']
    
    # Get the pixel area for the district
    pixel_count = district['properties']['Shape_Area']
    district_area_sq_km = pixel_count * pixel_area
    
    # Create a dictionary to store landcover areas for each time period
    landcover_areas_dict = {}
    
    # Loop through each time period
    for year in time_periods:
        # Get the land cover classification values for the district and year
        image_for_year = ee.Image('MODIS/006/MCD12Q1/{0}_01_01'.format(year)).select('LC_Type1')
        landcover_values = image_for_year.reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=district['geometry'],
            scale=Scale,
            maxPixels=1e13
        ).get('LC_Type1').getInfo()

        # Convert pixel counts to landcover areas in square kilometers
        landcover_areas = {landcover_names[str(landcover)]: count * pixel_area for landcover, count in landcover_values.items()}
        
        # Add landcover areas for the current year to the dictionary
        landcover_areas_dict[str(year)] = landcover_areas
    
    # Append the data to the list
    data.append({
        'Parent Region': parent_region_name,
        'District': district_name,
        **landcover_areas_dict,
        'Area (sq km)': district_area_sq_km
    })

# Create a DataFrame from the list of data
district_landcover_df = pd.DataFrame(data)

# Print the resulting DataFrame
print(district_landcover_df)


NameError: name 'pixel_area' is not defined

In [62]:
import pandas as pd

# ... (rest of your code to create the district_landcover_df DataFrame) ...

# Specify the path to the CSV file
csv_file_path = 'district_landcover_data2001_2020.csv'

# Export the DataFrame to a CSV file
district_landcover_df.to_csv(csv_file_path, index=False)

print("CSV file exported successfully.")


CSV file exported successfully.


In [64]:
import pandas as pd

# ... (rest of your code to create the district_landcover_df DataFrame) ...

# Reshape the DataFrame to a long format
melted_df = district_landcover_df.melt(
    id_vars=['Parent Region', 'District', 'Area (sq km)'],
    value_vars=[col for col in district_landcover_df.columns if col not in ['Parent Region', 'District', 'Area (sq km)']],
    var_name='Landcover',
    value_name='Area (sq km)'
)

# Specify the path to the Excel file
excel_file_path = 'melted_district_landcover_data.xlsx'

# Export the reshaped DataFrame to an Excel file
melted_df.to_excel(excel_file_path, index=False)

print("Excel file exported successfully.")


Excel file exported successfully.


In [65]:
import pandas as pd

# ... (rest of your code to create the district_landcover_df DataFrame) ...

# Initialize an empty list to store the data
data = []

# Iterate through the district features
for district in fvLayer2.getInfo()['features']:
    # Extract district and parent region names
    district_name = district['properties']['ADM2_NAME']
    parent_region_name = district['properties']['ADM1_NAME']
    
    # Get the pixel area for the district
    pixel_count = district['properties']['Shape_Area']
    district_area_sq_km = pixel_count * pixel_area
    
    # Get the land cover classification values for the district
    landcover_values = first_image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=district['geometry'],
        scale=Scale,
        maxPixels=1e13
    ).get('LC_Type1').getInfo()

    # Convert pixel counts to landcover areas in square kilometers
    landcover_areas = {landcover_names[str(landcover)]: count * pixel_area for landcover, count in landcover_values.items()}

    # Append the data to the list
    data.append({
        'Parent Region': parent_region_name,
        'District': district_name,
        **landcover_areas,
        'Area (sq km)': district_area_sq_km
    })

# Create a DataFrame from the list of data
district_landcover_df = pd.DataFrame(data)

# Print the resulting DataFrame
print(district_landcover_df)

# Specify the path to the Excel file
excel_file_path = 'district_landcover_data.xlsx'

# Export the DataFrame to an Excel file
district_landcover_df.to_excel(excel_file_path, index=False)

print("Excel file exported successfully.")


                             Parent Region             District   Grasslands  \
0   Cordillera Administrative region (CAR)                 Abra   269.162745   
1   Cordillera Administrative region (CAR)               Apayao    98.129412   
2   Cordillera Administrative region (CAR)              Benguet    34.700980   
3   Cordillera Administrative region (CAR)               Ifugao    29.352941   
4   Cordillera Administrative region (CAR)              Kalinga   117.802941   
5   Cordillera Administrative region (CAR)    Mountain Province    24.436275   
6            National Capital region (NCR)  Metropolitan Manila     8.079412   
7                 Region I (Ilocos region)         Ilocos Norte   288.562745   
8                 Region I (Ilocos region)           Ilocos Sur   232.082353   
9                 Region I (Ilocos region)             La Union   174.137255   
10                Region I (Ilocos region)           Pangasinan   432.803922   
11              Region II (Cagayan Valle

Evergreen Needleleaf Forests": The area covered by this landcover category is approximately 163.22 square meters.
"Grasslands": The area covered by grasslands is quite extensive, around 61,446.60 square meters.
"Permanent Wetlands": The area of permanent wetlands is around 33,674.17 square meters.
"Croplands": Croplands occupy an area of approximately 80,969.74 square meters.
"Urban and Built-up Lands": The area covered by urban and built-up lands is around 12,444.62 square meters.
"Cropland/Natural Vegetation Mosaics": This category covers a substantial area of approximately 128,058.53 square meters.
"Permanent Snow and Ice": The area of permanent snow and ice is a very small portion, about 0.33 square meters.
"Barren": Barren lands cover an area of around 187.04 square meters.
"NaN": This seems to be an undefined or missing landcover category, occupying about 24,426.31 square meters.

In [19]:
# Initialize an empty list to store the data
data = []

# List of time periods (years)
time_periods = [2001, 2005, 2010, 2015, 2020]

# Iterate through the district features
for district in fvLayer2.getInfo()['features']:
    # Extract district and parent region names
    district_name = district['properties']['ADM2_NAME']
    parent_region_name = district['properties']['ADM1_NAME']
    
    # Get the pixel area for the district
    pixel_count = district['properties']['Shape_Area']
    district_area_sq_km = pixel_count * pixel_area
    
    # Create a dictionary to store landcover areas for each time period
    landcover_areas_dict = {}
    
    # Loop through each time period
    for year in time_periods:
        # Get the land cover classification values for the district and year
        image_for_year = ee.Image('MODIS/006/MCD12Q1/{0}_01_01'.format(year)).select('LC_Type1')
        landcover_values = image_for_year.reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=district['geometry'],
            scale=Scale,
            maxPixels=1e13
        ).get('LC_Type1').getInfo()

        # Convert pixel counts to landcover areas in square kilometers
        landcover_areas = {landcover_names[str(landcover)]: count * pixel_area for landcover, count in landcover_values.items()}
        
        # Add landcover areas for the current year to the dictionary
        landcover_areas_dict.update({f'{year}_{landcover}': area for landcover, area in landcover_areas.items()})
    
    # Append the data to the list
    data.append({
        'Parent Region': parent_region_name,
        'District': district_name,
        **landcover_areas_dict,
        'Area (sq km)': district_area_sq_km
    })

# Create a DataFrame from the list of data
district_landcover_df = pd.DataFrame(data)

# Print the resulting DataFrame
print(district_landcover_df)

# Specify the path to the Excel file
excel_file_path = 'district_landcover_data.xlsx'

# Export the DataFrame to an Excel file
district_landcover_df.to_excel(excel_file_path, index=False)

print("Excel file exported successfully.")


NameError: name 'pixel_area' is not defined

In [40]:
# Import necessary libraries
import pandas as pd
from fuzzywuzzy import fuzz, process

# Load the district_landcover_data.xlsx file
district_landcover_df = pd.read_csv('12001_2020district_landcover_data.csv')


district_landcover_df.columns


Index(['Parent Region', 'District', 'Pop_2000', 'Pop_2010', 'Pop_2015',
       '2001_Grasslands', '2001_Croplands', '2001_Urban and Built-up Lands',
       '2001_Barren', '2001_Your Landcover Name for 17',
       '2001_Evergreen Broadleaf Forests', '2001_Deciduous Broadleaf Forests',
       '2001_Woody Savannas', '2001_Savannas', '2005_Grasslands',
       '2005_Croplands', '2005_Urban and Built-up Lands', '2005_Barren',
       '2005_Your Landcover Name for 17', '2005_Evergreen Broadleaf Forests',
       '2005_Deciduous Broadleaf Forests', '2005_Mixed Forests',
       '2005_Woody Savannas', '2005_Savannas', '2010_Grasslands',
       '2010_Croplands', '2010_Urban and Built-up Lands', '2010_Barren',
       '2010_Your Landcover Name for 17', '2010_Evergreen Broadleaf Forests',
       '2010_Deciduous Broadleaf Forests', '2010_Mixed Forests',
       '2010_Woody Savannas', '2010_Savannas', '2015_Grasslands',
       '2015_Croplands', '2015_Urban and Built-up Lands',
       '2015_Cropland/Natur

In [57]:
pip install geopandas geoplot


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 3.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 258.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 570.5 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
import ee
import geemap

# Authenticate and initialize Earth Engine
# ee.Authenticate()
# ee.Initialize()

# Load the GAUL dataset
gaul = ee.FeatureCollection("FAO/GAUL/2015/level1")

# Define the regions to highlight
highlighted_regions = ['National Capital Region', 'Central Luzon', 'Calabarzon']

# Filter the GAUL dataset to get the highlighted regions
highlighted_regions_gaul = gaul.filter(ee.Filter.inList('ADM1_NAME', highlighted_regions))

# Create a map centered on the Philippines
Map = geemap.Map(center=[12, 122], zoom=6)

# Add the GAUL dataset to the map
Map.addLayer(highlighted_regions_gaul, {}, 'Highlighted Regions')

# Set the basemap to ROADMAP
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map.add_basemap('ROADMAP')

# Show the map
Map

Map(center=[12, 122], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [38]:
import ee
import geemap



# Load the GAUL dataset
gaul = ee.FeatureCollection("FAO/GAUL/2015/level1")

# Define the regions to highlight
highlighted_regions = ['National Capital Region', 'Central Luzon', 'Calabarzon']

# Filter the GAUL dataset to get the highlighted regions
highlighted_regions_gaul = gaul.filter(ee.Filter.inList('ADM1_NAME', highlighted_regions))

# Create a map centered on the Philippines
Map = geemap.Map(center=[12, 122], zoom=6)



# Set the basemap to ROADMAP
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map.add_basemap(basemap=Map)
Map.add_layer_control()
# Add the GAUL dataset to the map
Map.addLayer(highlighted_regions_gaul, {}, 'Highlighted Regions')
# Show the map
Map


Map(center=[12, 122], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [25]:
!pip install earthengine-api



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


## Part 2 Timeseries Analysis


In [53]:
df.columns


Index(['Parent Region', 'District', 'Pop_2000', 'Pop_2010', 'Pop_2015',
       '2001_Grasslands', '2001_Croplands', '2001_Urban and Built-up Lands',
       '2001_Barren', '2001_Your Landcover Name for 17',
       '2001_Evergreen Broadleaf Forests', '2001_Deciduous Broadleaf Forests',
       '2001_Woody Savannas', '2001_Savannas', '2005_Grasslands',
       '2005_Croplands', '2005_Urban and Built-up Lands', '2005_Barren',
       '2005_Your Landcover Name for 17', '2005_Evergreen Broadleaf Forests',
       '2005_Deciduous Broadleaf Forests', '2005_Mixed Forests',
       '2005_Woody Savannas', '2005_Savannas', '2010_Grasslands',
       '2010_Croplands', '2010_Urban and Built-up Lands', '2010_Barren',
       '2010_Your Landcover Name for 17', '2010_Evergreen Broadleaf Forests',
       '2010_Deciduous Broadleaf Forests', '2010_Mixed Forests',
       '2010_Woody Savannas', '2010_Savannas', '2015_Grasslands',
       '2015_Croplands', '2015_Urban and Built-up Lands',
       '2015_Cropland/Natur

### Add pretty basemap

```Map.add_basemap('Esri.NatGeoWorldMap')```

basemaps = geemap.ee_basemaps.keys()

```print(basemaps)```

One final look at the map of the study area 







In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load your data into the DataFrame (district_landcover_df)
# ...

# List of regions of interest
regions_of_interest = ['Cordillera Administrative region (CAR)',
                      'National Capital region (NCR)',
                      'Region I (Ilocos region)',
                      'Region II (Cagayan Valley)',
                      'Region III (Central Luzon)',
                      'Region IV (Southern Tagalog)',
                      'Region IV-A (Calabarzon)',
                      'Region V (Bicol region)',
                      'Region VIII (Eastern Visayas)']

# Filter data for urban land areas in all time periods
filtered_data = district_landcover_df[
    (district_landcover_df['Parent Region'].isin(regions_of_interest)) &
    (district_landcover_df['District'] == 'Urban and Built-up Lands')
]

# Create a color palette for the regions
region_palette = sns.color_palette("Set1", n_colors=len(regions_of_interest))

# Create line plots for each region
plt.figure(figsize=(12, 8))

for idx, region in enumerate(regions_of_interest):
    region_data = filtered_data[filtered_data['Parent Region'] == region]
    plt.plot(region_data.columns[2:-1], region_data.iloc[0, 2:-1].astype(float), label=region, marker='o')

plt.title("Urban Landcover Transition Over Time")
plt.xlabel("Year")
plt.ylabel("Land Area")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()

# Display the plot
plt.show()


IndexError: single positional indexer is out-of-bounds

<Figure size 1200x800 with 0 Axes>

In [15]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.foliumap as emap
except:
    import geemap as emap

# Authenticate and initialize Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


In [34]:
import geemap
import geemap.foliumap as geemap
import seaborn as sns
import random
Map= geemap.Map(selected2, zoom=6)

Map.add_basemap('Esri.NatGeoWorldMap')
# vis_params = {
#     'min': 0,
#     'max': 100,
#     'opacity':1
#   #'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
# }


vis = {
    'bands': ['LC_Type1'],
    'min': 1.0,
    'max': 17.0,
    'palette': ['05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'],}
Scale=500

fvLayer2 = ee.FeatureCollection("FAO/GAUL/2015/level2").filterBounds(selected2)

# fvLayerSelected = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(
# ee.Filter.inList('ADM1_NAME'== list['HARYANA', 'DELHI']))

# Map.add_layer(fvLayerSelected, vis_params, 'fvLayerSelected')

Map.add_layer( fvLayer.filterBounds(selected2), vis_params,'fvLayer')

Map.add_layer(first_image,  vis, 'Landcover')

Map.add_layer(fvLayer2,  vis_params,'fvLayer2')

Map.addLayerControl()
Map


TypeError: __init__() takes 1 positional argument but 2 were given

## Export Results as Shapefile

In [ ]:
out_shp= os.path.join(out_dir, 'states.shp')
geemap.ee_export_vector(fvLayer.filterBounds(selected2), out_shp, verbose=True)

Generating URL ...
Please wait ...
Data downloaded to /Users/rishi/DownloadsDec29/states.shp


In [ ]:
out_shp= os.path.join(out_dir, 'states.shp')
geemap.ee_export_vector(fvLayer2.filterBounds(selected2), out_shp, verbose=True)

Generating URL ...
Please wait ...
Data downloaded to /Users/rishi/DownloadsDec29/states.shp


Having imported the shapefiles of districts and states, we can now overlay them over the landcover map which we have. Then we will calculate the change in landcover over time by district.

## Viewing the raster and generating landcover classification results as .csv in the directory

In [ ]:
## State Level
out_csv= os.path.join(out_dir, 'output.csv')
geemap.zonal_statistics(first_image.clipToBoundsAndScale(selected2), fvLayer.filterBounds(selected2), out_csv,  statistics_type='MEAN')


Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/rishi/DownloadsDec29/output.csv


In [ ]:
## District Level
out_csv= os.path.join(out_dir, 'CombinedCountMeanDistrict.csv')
geemap.zonal_statistics( first_image.clipToBoundsAndScale(selected2), fvLayer2.filterBounds(selected2), out_csv,  statistics_type='COMBINED_COUNT_MEAN')



Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/rishi/DownloadsDec29/CombinedCountMeanDistrict.csv


In [ ]:
out_csv= os.path.join(out_dir, 'outputLandcoverNCRDistrict.csv')
igbpLandCoverVis = {
  'min': 1.0,
  'max': 17.0,
  'palette': [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ],
}


Map=geemap.Map()

Map.addLayer(first_image, igbpLandCoverVis, 'first_image')

districts= fvLayer2
geemap.zonal_statistics_by_group(first_image.clipToBoundsAndScale(selected2), districts.filterBounds(selected2), out_csv)

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to /Users/rishi/DownloadsDec29/outputLandcoverNCRDistrict.csv


In [ ]:
import pandas as pd
# with open("outputDistrict.csv", 'r') as csvfile:
#     csvreader= csv.reader(csvfile)
    
pd.read_csv(out_csv, index_col=0)

,Class_13,Class_14,Class_9,Class_10,Class_sum,ADM1_CODE,ADM2_NAME,EXP2_YEAR,ADM1_NAME,DISP_AREA,Shape_Leng,STATUS,STR2_YEAR,ADM0_NAME,ADM0_CODE,Shape_Area,ADM2_CODE,system:index
Class_12,,,,,,,,,,,,,,,,,,
1422591118,38433153,0,0,68919451,1.529944e+09,1492,Mahendragarh,3000,Haryana,NO,3.593427,Member State,1990,India,115,0.177959,17662,0001000000000000124c
2155022223,21059397,18724400,18724400,0,2.213530e+09,1492,Sonepat,3000,Haryana,NO,2.530853,Member State,1990,India,115,0.205076,17665,0001000000000000124f
977352239,55913734,0,11115983,0,1.044382e+09,1492,Panipat,3000,Haryana,NO,1.713028,Member State,1990,India,115,0.113767,70136,00010000000000001255
1232288063,18848018,0,0,0,1.251136e+09,1492,Rewari,3000,Haryana,NO,2.280312,Member State,1990,India,115,0.115190,70137,00010000000000001256
14388184,0,0,0,0,1.438818e+07,1506,Bharatpur,3000,Rajasthan,NO,6.173368,Member State,1990,India,115,0.460272,17838,0001000000000000129b
71417889,0,0,0,0,7.141789e+07,1506,Jaipur,3000,Rajasthan,NO,7.268674,Member State,1996,India,115,1.022299,17847,000100000000000012a4
2530731660,144495870,0,0,0,2.675228e+09,70081,Aligarh,3000,Uttar Pradesh,NO,4.196777,Member State,1997,India,115,0.354363,17889,000100000000000006d1
571169482,6593318,0,0,0,5.777628e+08,70081,Mathura,3000,Uttar Pradesh,NO,3.181124,Member State,1997,India,115,0.308418,17925,000100000000000006f0
26119544,0,0,0,0,2.611954e+07,70081,Moradabad,3000,Uttar Pradesh,NO,3.797381,Member State,1997,India,115,0.311935,17928,000100000000000006f3


## Part 3 Calculate Zonal Statistics by group
Calculate the landcover change in each district and state since 2001.    

In [ ]:
igbpLandCoverVis = {
  'min': 1.0,
  'max': 17.0,
  'palette': [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ],
}


Map=geemap.Map()

Map.addLayer(first_image, igbpLandCoverVis, 'first_image')

districts= fvLayer2
geemap.zonal_statistics_by_group(first_image.clipToBoundsAndScale(selected2), districts, )

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to /Users/rishi/zonal_stats_by_group.csv


In [ ]:
districts= fvLayer2
geemap.zonal_statistics_by_group(first_image.clipToBoundsAndScale(selected2), districts.filterBounds(selected2), out_csv, statistics_type='SUM')

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to /Users/rishi/DownloadsDec29/outputLandcoverNCRDistrict.csv


In [ ]:
### Display of variables

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pandas as pd
import pprint
har= pd.read_csv('/Users/rishi/DownloadsDec29/CombinedCountMeanDistrict.csv')

from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

#Rename columns
har= pd.DataFrame(har)
har= har.rename(columns={
    'ADM1_CODE': 'State Code',
    'ADM2_NAME': 'District',
    'ADM1_NAME': 'State',
    'ADM0_NAME': "Country",
    'ADM0_CODE': 'Country Code',
    'ADM2_CODE':'District Code',
    "Class_12": 'Croplands',
    'Class_13': 'Urban Built Up Lands',
    'Class_14': 'Cropland/Natural Vegetation Mosiacs',
    'Class_9': ' Savannas',
    'Class_10': 'Grasslands'
    
 })


## Add the land cover classes and replace the column heading from csv file

# classes_list = ['Evergreen Needleleaf Forests', 'Evergreen Broadleaf Forests', 
#   'Deciduous Needleleaf Forests', 'Deciduous Broadleaf Forests', 
#   'Mixed Forests', 'Closed Shrublands', 'Open Shrublands', 'Woody Savannas',
#   'Savannas', 'Grasslands', 'Permanent Wetlands', 'Croplands', 'Urban and Built-up Lands', 
#   'Cropland, Natural Vegetation, Mosaic', 'Permanent Snow and Ice', 'Barren', 'Water Bodies'];

LCvars = har.columns.to_list()


LCvars


['count',
 'mean',
 'State Code',
 'District',
 'EXP2_YEAR',
 'State',
 'DISP_AREA',
 'Shape_Leng',
 'STATUS',
 'STR2_YEAR',
 'Country',
 'Country Code',
 'Shape_Area',
 'District Code',
 'system:index']

The aim here is to build a matrix which explains the temporal landcover change (Y- Axis is greenspace per unit of urban land, X- Axis is time) of each district in the study area(Line Color)
    Since we are interested in knowing the change in green cover (forests, savannas and cultivated areas included) vis-a-vis built area over time, it could be a good idea to transpoose the matrix and have names of districts as columns and 
    For further analysis, the pixels in this classification will then be assigned their individual weight depending upon 1. the ecosystem condition and 2. its ability to deliver ecosystem services in the area.
    **Use Pandas Profiling to generate report for each variable. It gives a detailed report on the accuracy of each variable. 
    https://pandas-profiling.ydata.ai/examples/master/meteorites/meteorites_report.html#pp_var_-2808404728303126642**

In [ ]:
# ### Work with pandas
# make a line graph of all the districts
# add more data from other variables
# import pandas as pd
# har['TotalGreenCover']= har['Croplands']+har['Cropland/Natural Vegetation Mosiacs']+har[' Savannas']+har['Grasslands']
# har['UrbRatio']= har['TotalGreenCover']/har['Urban Built Up Lands']

In [ ]:
#Reorder the column

In [ ]:
from jinja2 import Environment, PackageLoader, select_autoescape
env = Environment(loader=PackageLoader("pandas"),
                  autoescape=select_autoescape())
from pandas_profiling import ProfileReport

In [ ]:
#Code for preparing summary of included variables. Use Pandas profiling documentation which is quite elaborate
# profile= ProfileReport(har, title='Report-- ')
# profile.to_widgets()
# profile.to_notebook_iframe()